Basic Symmetric Encryption


In [5]:
from cryptography.fernet import Fernet
import cryptography
import os
from cryptography.fernet import Fernet, InvalidToken

def key_generation():
    key = Fernet.generate_key()
    with open("Secret.key", "wb") as key_file:
        key_file.write(key)

def load_key():
    return open("Secret.key", "rb").read()

def encrypt(filename, key):
    f = Fernet(key)
    with open(filename, "rb") as file:
        file_data = file.read()
        encrypted_data = f.encrypt(file_data)
    with open(filename, "wb") as file:
        file.write(encrypted_data)

def decrypt(filename, key):
    f = Fernet(key)
    with open(filename, "rb") as file:
        encrypted_data = file.read()
        try:
            decrypted_data = f.decrypt(encrypted_data)
        except InvalidToken:
            print("Invalid key")
            return
    with open(filename, "wb") as file:
        file.write(decrypted_data)

choice = input("Enter E to encrypt and D to decrypt: ").lower()
if choice == 'e':
    filename = input("Enter the filename to encrypt (include file extension): ")
    if os.path.exists(filename):
        # Check if key exists; if not, generate a new one
        if not os.path.exists("Secret.key"):
            key_generation()
        key = load_key()
        encrypt(filename, key)
        print("File encrypted successfully!!")
    else:
        print("File not found, enter a different filename and try again!")

elif choice == 'd':
    filename = input("Enter the filename to decrypt (include file extension): ")
    if os.path.exists(filename):
        key = load_key()
        decrypt(filename, key)
        print("File decrypted successfully!!")
    else:
        print("File not found, enter a different filename and try again!")

else:
    print("Invalid choice, enter E to encrypt the file and D to decrypt the file.")


Enter E to encrypt and D to decrypt: e
Enter the filename to encrypt (include file extension): username.txt
File encrypted successfully!!


AES Algorithm

In [6]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

# Function to generate AES key from password
def generate_key(password: str) -> bytes:
    return password.encode('utf-8').ljust(32)[:32]  # Ensure key is 32 bytes long

# Encrypting the file
def encrypt_file(filename: str, password: str):
    key = generate_key(password)
    iv = os.urandom(16)  # Generate a random IV
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Read the original file
    with open(filename, 'rb') as file:
        file_data = file.read()

    # Pad the data to be a multiple of block size
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(file_data) + padder.finalize()

    # Encrypt the padded data
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Write the IV and the encrypted data to the file
    with open(filename, 'wb') as file:
        file.write(iv + encrypted_data)

    return "File encrypted successfully!"

# Decrypting the file
def decrypt_file(filename: str, password: str):
    key = generate_key(password)

    # Read the encrypted file
    with open(filename, 'rb') as file:
        if os.path.getsize(filename) < 16:
            return "File is too small to be encrypted."

        iv = file.read(16)  # Read the first 16 bytes for IV
        encrypted_data = file.read()

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the data
    decrypted_padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Unpad the decrypted data
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    decrypted_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()

    # Write the decrypted data back to the file
    with open(filename, 'wb') as file:
        file.write(decrypted_data)

    return "File decrypted successfully!"


# Create UI components
title = widgets.HTML("<h2 style='color: white;'>AES File Encryptor/Decryptor</h2>")
instructions = widgets.HTML("<p>Upload a file or enter the filename (including extension) and a password. Select the action (Encrypt or Decrypt) and click 'Execute'.</p>")
file_upload = widgets.FileUpload(accept='*', multiple=False)
file_input = widgets.Text(placeholder='Enter filename (include extension)')
password_input = widgets.Password(placeholder='Enter password')

# Action buttons
encrypt_button = widgets.Button(description='Encrypt', layout=widgets.Layout(width='200px', height='50px'))
decrypt_button = widgets.Button(description='Decrypt', layout=widgets.Layout(width='200px', height='50px'))
reset_button = widgets.Button(description='Reset', layout=widgets.Layout(width='200px', height='50px'))

# Button styling using custom CSS
button_style = {
    'background_color': 'black',
    'color': 'white',
    'font_weight': 'bold',
    'font_size': '16px'
}

# Apply the style to the buttons
def style_button(button):
    button.style.button_color = button_style['background_color']
    button.style.font_weight = button_style['font_weight']
    button.style.font_size = button_style['font_size']

style_button(encrypt_button)
style_button(decrypt_button)
style_button(reset_button)

# Organize buttons into two rows using HBox and VBox
hbox1 = widgets.HBox([encrypt_button, decrypt_button])  # First row of buttons
hbox2 = widgets.HBox([ reset_button])  # Second row of buttons

# Stack the HBoxes in a VBox to arrange buttons vertically
vbox = widgets.VBox([hbox1, hbox2])

output = widgets.Output()

def on_file_upload(change):
    if file_upload.value:
        file_input.value = list(file_upload.value.keys())[0]

file_upload.observe(on_file_upload, names='value')

def on_action_button_click(action):
    with output:
        clear_output()
        try:
            filename = file_input.value or list(file_upload.value.keys())[0]
        except IndexError:
            print("No file selected. Please upload a file or enter a filename.")
            return

        password = password_input.value

        if not password:
            print("Please enter a password.")
            return

        print("Processing... Please wait.")

        if os.path.exists(filename):
            if action == 'Encrypt':
                message = encrypt_file(filename, password)
            else:
                message = decrypt_file(filename, password)
        else:
            message = "File not found. Please check the filename."

        clear_output()  # Clear previous output
        print(message)  # Display the message

def reset_fields(b):
    global file_upload
    # Clear the uploaded file
    if file_upload.value:
        filename = list(file_upload.value.keys())[0]
        if os.path.exists(filename):
            os.remove(filename)  # Delete the uploaded file

    # Recreate the file upload widget to reset its state
    file_upload = widgets.FileUpload(accept='*', multiple=False)
    file_upload.observe(on_file_upload, names='value')

    # Reset input fields
    file_input.value = ''  # Clear the filename input
    password_input.value = ''  # Clear the password input
    output.clear_output()  # Clear the output area
    file_input.placeholder = 'Enter filename (include extension)'  # Update placeholder

# Button click handlers
encrypt_button.on_click(lambda b: on_action_button_click('Encrypt'))
decrypt_button.on_click(lambda b: on_action_button_click('Decrypt'))
reset_button.on_click(reset_fields)

# Display UI
display(title, instructions, file_upload, file_input, password_input, vbox, output)


HTML(value="<h2 style='color: white;'>AES File Encryptor/Decryptor</h2>")

HTML(value="<p>Upload a file or enter the filename (including extension) and a password. Select the action (En…

FileUpload(value={}, accept='*', description='Upload')

Text(value='', placeholder='Enter filename (include extension)')

Password(placeholder='Enter password')

Output()